In [19]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from keras import Model
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, Conv2D, ReLU, MaxPooling2D, Flatten, Activation
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing import image
from keras.models import load_model
from keras.models import Sequential
from keras.initializers import GlorotNormal
import tensorflow as tf
from keras.regularizers import l2
from sklearn.metrics import classification_report

In [4]:
!cp "/content/drive/MyDrive/15 GB Storage 1/ML Lab/Data/Segment_output_final4.zip" Segment_output_final4.zip

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "Segment_output_final4.zip"

In [20]:
#Size settings
img_size = (224, 224)
num_classes = 32
batch_size = 32

In [21]:

#Model
base_model = EfficientNetB0(weights=None, include_top=False, input_shape=img_size+(3,))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)
x = Dense(128, activation='relu', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(32, activation='softmax', kernel_initializer='he_normal')(x)
model = Model(inputs=base_model.input, outputs=output)
model.summary()

for layer in model.layers:
   layer.trainable = True

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 224, 224, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_1[0][0]            
____________________________________________________________________________________________

In [22]:
model.load_weights('/content/drive/MyDrive/15 GB Storage 1/ML Lab/Checkpoint/EfficientNetB0/EfficientNetB0')

In [ ]:
model=load_model('/content/drive/MyDrive/15 GB Storage 1/ML Lab/Models/EfficientNetB0/EfficientNetB0_27_3_2021.h5')

In [23]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.3,
        horizontal_flip=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '/content/Segment_output_final4/train',
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical")
validation_generator = test_datagen.flow_from_directory(
        '/content/Segment_output_final4/validation',
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical")
test_generator = test_datagen.flow_from_directory(
        '/content/Naver_test_segment_output2/test',
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False)

Found 22661 images belonging to 32 classes.
Found 6659 images belonging to 32 classes.
Found 2855 images belonging to 32 classes.


In [25]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [30]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.0000001, verbose=1)
my_callback = [ModelCheckpoint(filepath='/content/drive/MyDrive/15 GB Storage 1/ML Lab/Checkpoint/EfficientNetB0', verbose=2, save_best_only=True, save_weights_only=True),reduce_lr]
model_history = model.fit(train_generator, validation_data=validation_generator, epochs=100, verbose=1, callbacks=my_callback)

Epoch 1/100
709/709 [==============================] - 317s 447ms/step - loss: 0.2535 - accuracy: 0.9307 - val_loss: 0.2135 - val_accuracy: 0.9387

Epoch 00001: val_loss improved from inf to 0.21355, saving model to /content/drive/MyDrive/15 GB Storage 1/ML Lab/Checkpoint/EfficientNetB0
Epoch 2/100
304/709 [===========>..................] - ETA: 2:52 - loss: 0.2823 - accuracy: 0.9241

KeyboardInterrupt: ignored

In [34]:
score = model.evaluate(test_generator, verbose=1)
print('Loss:', score[0])
print('Accuracy', score[1])

90/90 [==============================] - 6s 66ms/step - loss: 2.2547 - accuracy: 0.4396
Loss: 2.2546815872192383
Accuracy 0.43957969546318054


In [ ]:
predict = model.predict(test_generator)
y_test = test_generator.classes
y_predict = np.argmax(predict, axis=1)
print(classification_report(y_test, y_predict))

In [35]:
model.save('/content/drive/MyDrive/15 GB Storage 1/ML Lab/Models/EfficientNetB0/EfficientNetB0_1_4_2021.h5')

In [ ]:
#Cheating colab afk detection
while True:pass


#Or paste this to the chrome terminal
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);